<a href="https://colab.research.google.com/github/DavidSchneider47/ALIrecommendation/blob/main/Census_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip install geopandas

In [18]:
import geopandas as gpd

In [19]:
transit_areas = gpd.read_file('buffered_stations.geojson')

In [20]:
transit_areas.head()

,Lon,Lat,Facility N,Facility T,Primary Mo,Street Add,City,State,geometry
0,-150.06188,62.18922,Talkeetna Depot,At-Grade Fixed Guideway Station,AR,Mile 13.7 Talkeetna Spur Rd.,Talkeetna,AK,"POLYGON ((-150.05827 62.18922, -150.05828 62.1..."
1,-149.58490,61.10283,Anchorage Airport Bill Sheffield Depot,Elevated Fixed Guideway Station,AR,4980 W Int'L Airport Rd.,Anchorage,AK,"POLYGON ((-149.58129 61.10283, -149.58130 61.1..."
2,-149.53262,61.13180,Anchorage Historic Depot (all inclusive),At-Grade Fixed Guideway Station,AR,401 - 431 1St. Ave.,Anchorage,AK,"POLYGON ((-149.52901 61.13180, -149.52902 61.1..."
3,-149.26326,60.07287,Seward Depot,At-Grade Fixed Guideway Station,AR,408 Port Ave.,Seward,AK,"POLYGON ((-149.25965 60.07287, -149.25966 60.0..."
4,-149.10306,60.56418,Girdwood Depot,At-Grade Fixed Guideway Station,AR,Toadstool Dr,Girdwood,AK,"POLYGON ((-149.09945 60.56418, -149.09946 60.5..."


In [21]:
import zipfile

In [22]:

zip_path = 'tl_2022_48_bg.zip'

In [23]:
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall()

In [24]:
shp_path = 'tl_2022_48_bg.shp'
block_groups = gpd.read_file(shp_path)

In [25]:
joined_data = gpd.sjoin(transit_areas, block_groups, how="inner", op="intersects")

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3473: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):
<ipython-input-25-056af29aa828>:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: EPSG:4269

  joined_data = gpd.sjoin(transit_areas, block_groups, how="inner", op="intersects")


In [26]:
import requests

In [103]:
# Define the base URL for the API
base_url = 'https://api.census.gov/data/2019/acs/acs5'

In [118]:
state_code = '48'  # Texas
county_code = '113'  # Dallas County
params = {
    'get': 'NAME,B01001_001E,B19013_001E',
    'for': 'block group:*',
    'in': 'state:{} county:{}'.format(state_code, county_code)
}

In [119]:
print(response.status_code)
print(response.text)

400
error: missing 'for' argument


In [120]:
# Make the request
response = requests.get(base_url, params=params)

In [121]:
# Parse the response as JSON
data = response.json()

In [122]:
import pandas as pd

# Convert the API data to a DataFrame
census_data = pd.DataFrame(data[1:], columns=data[0])

In [123]:
census_data['GEOID'] = census_data['state'] + census_data['county'] + census_data['tract']+ census_data['block group']

In [124]:
# Merge with the spatially joined data
final_data = pd.merge(joined_data, census_data, left_on='GEOID', right_on='GEOID')

In [125]:
final_data.head()

,Lon,Lat,Facility N,Facility T,Primary Mo,Street Add,City,State,geometry,index_right,...,AWATER,INTPTLAT,INTPTLON,NAME,B01001_001E,B19013_001E,state,county,tract,block group
0,-97.014677,32.736280,Hurst / Bell (TRE),Exclusive Platform Station,CR,3232 Bell Helicopter Blvd.,Hurst,TX,"POLYGON ((-97.01106 32.73628, -97.01108 32.735...",14804,...,0,+32.7328303,-097.0119530,"Block Group 2, Census Tract 160.01, Dallas Cou...",675,-666666666,48,113,016001,2
1,-97.014677,32.736280,Hurst / Bell (TRE),Exclusive Platform Station,CR,3232 Bell Helicopter Blvd.,Hurst,TX,"POLYGON ((-97.01106 32.73628, -97.01108 32.735...",14803,...,0,+32.7388920,-097.0209225,"Block Group 2, Census Tract 161, Dallas County...",1445,44716,48,113,016100,2
2,-97.014677,32.736280,Hurst / Bell (TRE),Exclusive Platform Station,CR,3232 Bell Helicopter Blvd.,Hurst,TX,"POLYGON ((-97.01106 32.73628, -97.01108 32.735...",6138,...,0,+32.7402613,-097.0115612,"Block Group 1, Census Tract 160.01, Dallas Cou...",1342,27500,48,113,016001,1
3,-97.006894,32.817236,West Irving Station,Exclusive Platform Station,CR,4200 Jackson St,Irving,TX,"POLYGON ((-97.00328 32.81724, -97.00330 32.816...",3069,...,3554,+32.8073333,-097.0252164,"Block Group 1, Census Tract 153.03, Dallas Cou...",1368,49792,48,113,015303,1
4,-97.006894,32.817236,West Irving Station,Exclusive Platform Station,CR,4200 Jackson St,Irving,TX,"POLYGON ((-97.00328 32.81724, -97.00330 32.816...",1914,...,0,+32.8202712,-096.9984525,"Block Group 1, Census Tract 144.08, Dallas Cou...",1757,63953,48,113,014408,1


In [126]:
from google.colab import files

In [127]:
final_data.to_csv('final_data.csv')

In [128]:
files.download('final_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [38]:
# Create a pivot table with Lon and Lat as indices, summing the B01001_001E column
pivot_data = final_data.pivot_table(values='B01001_001E',
                                    index=['Lon', 'Lat'],
                                    aggfunc='sum').reset_index()

In [39]:
# Merge the pivot_data with the original data to retain other columns
# We'll drop the B01001_001E column from the original data to avoid duplication
final_data_dropped = final_data.drop('B01001_001E', axis=1)
merged_data = pd.merge(final_data_dropped.drop_duplicates(subset=['Lon', 'Lat']), pivot_data, on=['Lon', 'Lat'])

In [40]:
# Display the first few rows of the merged data
merged_data.head()

,Lon,Lat,Facility N,Facility T,Primary Mo,Street Add,City,State,geometry,index_right,...,ALAND,AWATER,INTPTLAT,INTPTLON,NAME,state,county,tract,block group,B01001_001E
0,-97.014677,32.736280,Hurst / Bell (TRE),Exclusive Platform Station,CR,3232 Bell Helicopter Blvd.,Hurst,TX,"POLYGON ((-97.01106 32.73628, -97.01108 32.735...",14804,...,476424,0,+32.7328303,-097.0119530,"Block Group 2, Census Tract 160.01, Dallas Cou...",48,113,016001,2,67514451342
1,-97.006894,32.817236,West Irving Station,Exclusive Platform Station,CR,4200 Jackson St,Irving,TX,"POLYGON ((-97.00328 32.81724, -97.00330 32.816...",3069,...,5641649,3554,+32.8073333,-097.0252164,"Block Group 1, Census Tract 153.03, Dallas Cou...",48,113,015303,1,13681757627
2,-96.947459,32.814444,Downtown Irving/Heritage Crossing Station,Simple At-Grade Platform Station,CR,201 Rock Island Rd,Irving,TX,"POLYGON ((-96.94384 32.81444, -96.94386 32.814...",4698,...,826090,0,+32.8198675,-096.9557919,"Block Group 1, Census Tract 145.02, Dallas Cou...",48,113,014502,1,10981701
3,-96.935070,32.875538,Irving Convention Center Station - Grade,Exclusive Platform Station,LR,480 W Northwest Hwy,Irving,TX,"POLYGON ((-96.93146 32.87554, -96.93147 32.875...",2309,...,1044995,88399,+32.8889657,-096.9328849,"Block Group 1, Census Tract 141.38, Dallas Cou...",48,113,014138,1,1363
4,-96.925522,32.982252,Trinity Mills Station-Grade,Simple At-Grade Platform Station,LR,2525 Blanton Drive,Carrollton,TX,"POLYGON ((-96.92191 32.98225, -96.92193 32.981...",7167,...,2191761,57452,+32.9805065,-096.9115647,"Block Group 2, Census Tract 137.19, Dallas Cou...",48,113,013719,2,19592784


In [41]:
census_data.head()

,NAME,B01001_001E,state,county,tract,block group,GEOID
0,"Block Group 3, Census Tract 142.03, Dallas Cou...",1699,48,113,014203,3,481130142033
1,"Block Group 1, Census Tract 142.03, Dallas Cou...",712,48,113,014203,1,481130142031
2,"Block Group 1, Census Tract 149.02, Dallas Cou...",1126,48,113,014902,1,481130149021
3,"Block Group 2, Census Tract 149.02, Dallas Cou...",1659,48,113,014902,2,481130149022
4,"Block Group 4, Census Tract 172.01, Dallas Cou...",1296,48,113,017201,4,481130172014


In [42]:
from google.colab import files

In [43]:
census_data.to_csv('census_data.csv')

In [44]:
files.download('census_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [45]:
joined_data.head()

,Lon,Lat,Facility N,Facility T,Primary Mo,Street Add,City,State,geometry,index_right,...,TRACTCE,BLKGRPCE,GEOID,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON
855,-106.504534,31.776988,Oregon - Glory Rd,Simple At-Grade Platform Station,SR,2798 N Oregon,El Paso,TX,"POLYGON ((-106.50092 31.77699, -106.50094 31.7...",13014,...,001502,4,481410015024,Block Group 4,G5030,S,1014282,0,+31.7720124,-106.5045651
856,-106.504154,31.777452,Stanton-Cincinnati,Simple At-Grade Platform Station,SR,2626 N Stanton,El Paso,TX,"POLYGON ((-106.50054 31.77745, -106.50056 31.7...",13014,...,001502,4,481410015024,Block Group 4,G5030,S,1014282,0,+31.7720124,-106.5045651
857,-106.503548,31.777760,Baltimore,Simple At-Grade Platform Station,SR,214 E Baltimore Dr,El Paso,TX,"POLYGON ((-106.49993 31.77776, -106.49995 31.7...",13014,...,001502,4,481410015024,Block Group 4,G5030,S,1014282,0,+31.7720124,-106.5045651
858,-106.502385,31.774601,Oregon - Boston,Simple At-Grade Platform Station,SR,2400 N Oregon,El Paso,TX,"POLYGON ((-106.49877 31.77460, -106.49879 31.7...",13014,...,001502,4,481410015024,Block Group 4,G5030,S,1014282,0,+31.7720124,-106.5045651
859,-106.501839,31.774882,Stanton-Kerby,Simple At-Grade Platform Station,SR,2300 N Stanton,El Paso,TX,"POLYGON ((-106.49822 31.77488, -106.49824 31.7...",13014,...,001502,4,481410015024,Block Group 4,G5030,S,1014282,0,+31.7720124,-106.5045651


In [46]:
from google.colab import files

In [47]:

joined_data.to_csv('joined_data.csv', index=False)

In [48]:
files.download('joined_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [49]:
files.download('joined_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [50]:
import requests

In [65]:
# Define the base URL for the API (change if using a different dataset)
base_url = 'https://api.census.gov/data/2019/acs/acs5'

In [52]:

# Make the request
response = requests.get(base_url, params=params)

In [53]:
# Check if the request was successful
if response.status_code == 200:
    data = response.json()
else:
    print("Error:", response.text)
    data = None

In [54]:
import pandas as pd

# Convert the API data to a DataFrame
census_data = pd.DataFrame(data[1:], columns=data[0])

In [55]:
# Print the first few rows of the DataFrame
print(census_data.head())

                                                NAME B01001_001E state county  \
0  Block Group 3, Census Tract 142.03, Dallas Cou...        1699    48    113   
1  Block Group 1, Census Tract 142.03, Dallas Cou...         712    48    113   
2  Block Group 1, Census Tract 149.02, Dallas Cou...        1126    48    113   
3  Block Group 2, Census Tract 149.02, Dallas Cou...        1659    48    113   
4  Block Group 4, Census Tract 172.01, Dallas Cou...        1296    48    113   

    tract block group  
0  014203           3  
1  014203           1  
2  014902           1  
3  014902           2  
4  017201           4  


In [56]:
census_data.head()

,NAME,B01001_001E,state,county,tract,block group
0,"Block Group 3, Census Tract 142.03, Dallas Cou...",1699,48,113,014203,3
1,"Block Group 1, Census Tract 142.03, Dallas Cou...",712,48,113,014203,1
2,"Block Group 1, Census Tract 149.02, Dallas Cou...",1126,48,113,014902,1
3,"Block Group 2, Census Tract 149.02, Dallas Cou...",1659,48,113,014902,2
4,"Block Group 4, Census Tract 172.01, Dallas Cou...",1296,48,113,017201,4


In [86]:
# Define the base URL for the API (change if using a different dataset)
base_url1 = 'https://api.census.gov/data/2009/acs5'

In [87]:

# Make the request
response = requests.get(base_url1, params=params)

In [88]:

# Print out the status code and the raw response to inspect
response.status_code, response.text

(200,
 '[["NAME","B01001_001E","state","county","tract","block group"],\n["Block Group 4, Census Tract 78.13, Dallas County, Texas","847","48","113","007813","4"],\n["Block Group 5, Census Tract 78.13, Dallas County, Texas","1158","48","113","007813","5"],\n["Block Group 6, Census Tract 78.13, Dallas County, Texas","638","48","113","007813","6"],\n["Block Group 7, Census Tract 78.13, Dallas County, Texas","820","48","113","007813","7"],\n["Block Group 1, Census Tract 78.14, Dallas County, Texas","1196","48","113","007814","1"],\n["Block Group 2, Census Tract 78.14, Dallas County, Texas","2000","48","113","007814","2"],\n["Block Group 3, Census Tract 78.14, Dallas County, Texas","1208","48","113","007814","3"],\n["Block Group 1, Census Tract 78.15, Dallas County, Texas","3267","48","113","007815","1"],\n["Block Group 2, Census Tract 78.15, Dallas County, Texas","1780","48","113","007815","2"],\n["Block Group 3, Census Tract 78.15, Dallas County, Texas","1014","48","113","007815","3"],\n

In [89]:
state_code = '48'  # Texas
county_code = '113'  # Dallas County

params = {
    'get': 'NAME,B01001_001E',  # Replace with the codes for the data you want
    'for': 'block group:*',
    'in': 'state:{} county:{}'.format(state_code, county_code)
}

In [90]:

# Make the request
response = requests.get(base_url1, params=params)

In [91]:
# Parse the response as JSON
data = response.json()

In [92]:
import pandas as pd

# Convert the API data to a DataFrame
census_data1 = pd.DataFrame(data[1:], columns=data[0])

In [93]:
census_data1['GEOID'] = census_data1['state'] + census_data1['county'] + census_data1['tract']+ census_data1['block group']

In [94]:
# Merge with the spatially joined data
final_data1 = pd.merge(joined_data, census_data1, left_on='GEOID', right_on='GEOID')

In [96]:
final_data1.head()

,Lon,Lat,Facility N,Facility T,Primary Mo,Street Add,City,State,geometry,index_right,...,ALAND,AWATER,INTPTLAT,INTPTLON,NAME,B01001_001E,state,county,tract,block group
0,-97.014677,32.736280,Hurst / Bell (TRE),Exclusive Platform Station,CR,3232 Bell Helicopter Blvd.,Hurst,TX,"POLYGON ((-97.01106 32.73628, -97.01108 32.735...",14804,...,476424,0,+32.7328303,-097.0119530,"Block Group 2, Census Tract 160.01, Dallas Cou...",944,48,113,016001,2
1,-97.014677,32.736280,Hurst / Bell (TRE),Exclusive Platform Station,CR,3232 Bell Helicopter Blvd.,Hurst,TX,"POLYGON ((-97.01106 32.73628, -97.01108 32.735...",14803,...,546085,0,+32.7388920,-097.0209225,"Block Group 2, Census Tract 161, Dallas County...",1585,48,113,016100,2
2,-97.014677,32.736280,Hurst / Bell (TRE),Exclusive Platform Station,CR,3232 Bell Helicopter Blvd.,Hurst,TX,"POLYGON ((-97.01106 32.73628, -97.01108 32.735...",6138,...,410380,0,+32.7402613,-097.0115612,"Block Group 1, Census Tract 160.01, Dallas Cou...",1570,48,113,016001,1
3,-97.006894,32.817236,West Irving Station,Exclusive Platform Station,CR,4200 Jackson St,Irving,TX,"POLYGON ((-97.00328 32.81724, -97.00330 32.816...",3069,...,5641649,3554,+32.8073333,-097.0252164,"Block Group 1, Census Tract 153.03, Dallas Cou...",353,48,113,015303,1
4,-97.006894,32.817236,West Irving Station,Exclusive Platform Station,CR,4200 Jackson St,Irving,TX,"POLYGON ((-97.00328 32.81724, -97.00330 32.816...",1914,...,886849,0,+32.8202712,-096.9984525,"Block Group 1, Census Tract 144.08, Dallas Cou...",2657,48,113,014408,1


In [98]:
final_data1.to_csv('final_data1.csv')

In [99]:
files.download('final_data1.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>